### Analysis and visualisation of Bartlett results

In [ ]:
import glob
import pickle
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import string as stringp

bartlett = """One night two young men from Egulac went down to the river to hunt seals and while they were there it became foggy and calm. Then they heard war-cries, and they thought: "Maybe this is a war-party". They escaped to the shore, and hid behind a log. Now canoes came up, and they heard the noise of paddles, and saw one canoe coming up to them. There were five men in the canoe, and they said:
"What do you think? We wish to take you along. We are going up the river to make war on the people."
One of the young men said,"I have no arrows."
"Arrows are in the canoe," they said.
"I will not go along. I might be killed. My relatives do not know where I have gone. But you," he said, turning to the other, "may go with them."
So one of the young men went, but the other returned home.
And the warriors went on up the river to a town on the other side of Kalama. The people came down to the water and they began to fight, and many were killed. But presently the young man heard one of the warriors say, "Quick, let us go home: that man has been hit." Now he thought: "Oh, they are ghosts." He did not feel sick, but they said he had been shot.
So the canoes went back to Egulac and the young man went ashore to his house and made a fire. And he told everybody and said: "Behold I accompanied the ghosts, and we went to fight. Many of our fellows were killed, and many of those who attacked us were killed. They said I was hit, and I did not feel sick."
He told it all, and then he became quiet. When the sun rose he fell down. Something black came out of his mouth. His face became contorted. The people jumped up and cried.
He was dead."""

In [ ]:
# for pkl in glob.glob('combined_results_dict 02-06.pkl'): #('bartlett_pkls/all_results_dict.pkl'):
#     with open(pkl, 'rb') as f:
#         d = pickle.load(f)
#     print(d)

In [ ]:
import os
import pickle

# Path to the directory containing pickle files
directory_path = 'bartlett_data2'

# Initialize a dictionary to hold combined data
combined_data = {}
combined_data[1] = {}

# Function to load data from a pickle file
def load_pickle_data(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data

# Read and combine data from all pickle files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.pkl'):  # Ensures that we are reading only pickle files
        file_path = os.path.join(directory_path, filename)
        data = load_pickle_data(file_path)

        # Assuming the structure is as described: data[1]['Category'][temp] -> ['string']
        for category in ['Universe', 'Politics', 'Health', 'Sport', 'Technology', 'Nature']:
            if category not in combined_data[1]:
                combined_data[1][category] = {}
            for temp in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 1.5]:
                if temp not in combined_data[1][category]:
                    combined_data[1][category][temp] = []
                if temp == 0:
                    combined_data[1][category][temp].append(data[1][category][temp])
                else:
                    combined_data[1][category][temp].extend(data[1][category][temp])

# Print the combined data to check
d = combined_data

In [ ]:
def plot_wordclouds(results_dict, models, keys=['greedy', 0.25, 0.5, 0.75], exclusion_text=bartlett, flip=False):
    # Convert the exclusion text into a set of words for faster lookup, stripping punctuation and lowercasing
    exclusion_text_processed = exclusion_text.translate(str.maketrans(stringp.punctuation, ' '*len(stringp.punctuation))).lower()
    exclusion_text_processed += ' ap s'
    exclusion_words = set(exclusion_text_processed.split()) 
    
    # Flipping the rows and columns
    num_rows = len(keys)  # Now based on the number of keys
    num_cols = len(models)  # Now based on the number of models
    
    # Create a figure for the subplots with flipped dimensions
    if flip:
        fig, axs = plt.subplots(num_cols, num_rows, figsize=(num_cols * 5, num_rows * 5))
    else:
        fig, axs = plt.subplots(num_rows, num_cols, figsize=(num_cols * 5, num_rows * 5))
    
    # Ensure axs is always 2D array for consistent indexing
    if num_rows == 1 or num_cols == 1:
        axs = np.atleast_2d(axs)
    
    # Adjust layout
    plt.tight_layout(pad=3.0)
    
    def preprocess_text(text):
        text = text.translate(str.maketrans(stringp.punctuation, ' '*len(stringp.punctuation))).lower()
        return ' '.join([word for word in text.split() if word not in exclusion_words])
    
    # Function to determine color scheme based on model
    def get_color_func(model):
        return 'winter'
    
    # Iterate through each model and key to plot word clouds with flipped rows and columns
    for col, model in enumerate(models):
        for row, key in enumerate(keys):
            original_text = ' '.join([r[0:len(bartlett)+50] for r in results_dict[model].get(key, '')])
            text = preprocess_text(original_text)
            if len(text.split()) > 0:
                wordcloud = WordCloud(width=400, height=400, relative_scaling=0.5, normalize_plurals=False,
                                      max_font_size=60,
                                      background_color ='white', colormap=get_color_func(model)).generate(text)

                if flip:
                    axs_index = axs[col, row] 
                else:
                    axs_index = axs[row, col]
                axs_index.imshow(wordcloud, interpolation='bilinear')
                axs_index.axis('off')
                axs_index.set_title(f'{model} - {key}')
    
    # Show plot
    plt.savefig('wordcloud.png', dpi=500)
    plt.show()

plot_wordclouds(d[1], models=d[1].keys(), keys=[0.1, 0.5, 1.0, 1.5], flip=True)


In [ ]:
plot_wordclouds(d[1], models=d[1].keys(), keys=[0.1, 0.5, 1.0, 1.5])

In [ ]:
bartlett_words = set(bartlett.lower().split())

In [ ]:
import numpy as np
from scipy.stats import sem
import matplotlib.pyplot as plt

temps_subset = [0, 0.5, 1.0, 1.5]

# Example data calculation setup (uncomment and complete your previous loading and processing steps)
new_words_count = {category: {temp: [] for temp in temps_subset} for category in combined_data[1]}
for category, temps in combined_data[1].items():
    for temp, strings in temps.items():
        if temp in temps_subset:
            for string in strings:
                string_words = set(string[0:len(bartlett)].lower().split())
                new_words = string_words - bartlett_words
                new_words_count[category][temp].append(len(new_words))

# Setup plot
plt.figure(figsize=(5.5, 3))
colors = plt.cm.viridis(np.linspace(0, 1, len(new_words_count)))  # Colormap for differentiating categories

for i, (category, temps) in enumerate(new_words_count.items()):
    # Calculate average new words and SEM for each temperature in this category
    average_new_words = {}
    sem_new_words = {}
    for temp, words in temps.items():
        if temp in temps_subset:
            if words:
                average_new_words[temp] = np.mean(words)
                sem_new_words[temp] = sem(words)  # Calculate SEM here
            else:
                average_new_words[temp] = 0
                sem_new_words[temp] = 0

    # Prepare data for plotting
    temperatures = sorted(average_new_words.keys())
    average_counts = [average_new_words[temp] for temp in temperatures]
    error = [sem_new_words[temp] for temp in temperatures]  # SEM errors

    # Plotting with error bars
    plt.errorbar(temperatures, average_counts, yerr=error, fmt='-o', color=colors[i], label=category, capsize=5)

plt.xlabel('Temperature')
plt.ylabel('Mean no. new words')
#plt.title('New Words in Recalled Stories vs. Temperature by Category')
plt.legend(title='Category', ncol=2)
plt.savefig('tempnewwords.png', dpi=300, bbox_inches='tight')
plt.show()
